In [2]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok
!pip install face_alignment==1.2.0 redner-gpu
!pip install mediapipe
!pip install aspose-3d

In [1]:
!ngrok authtoken 2NV2FysIjrbVA8zQsXS5D0ClOsY_26hW6MQwoCDVvPMS7Gohu

Authtoken saved to configuration file: C:\Users\HP/.ngrok2/ngrok.yml


In [1]:
from flask import Flask,render_template,request,redirect
from flask_ngrok import run_with_ngrok
import sys
import os
from os.path import join, dirname, realpath
from werkzeug.utils import secure_filename
import cv2
import tensorflow as tf
from keras.preprocessing import image
from keras.models import load_model,model_from_json
import numpy as np
import jinja2
import aspose.threed as a3d
from optimizer import Optimizer
from config import Config
config = Config()
config.fillFromDicFile('C:/Users/HP/NextFace/optimConfig.ini')
config.device = 'cpu'
config.path = 'C:/Users/HP/NextFace/baselMorphableModel/'
UPLOAD_FOLDER = 'C:/Users/HP/3DEmotion/static/'
outputDir = 'C:/Users/HP/NextFace/static/output/' 
optimizer = Optimizer(outputDir ,config)

loading optim config from:  C:/Users/HP/NextFace/optimConfig.ini
Loading Basel Face Model 2017 from C:/Users/HP/NextFace/baselMorphableModel//morphableModel-2017.pickle...
loading mesh normals...
loading uv parametrization...
loading landmarks association file...
creating sampler...


C:\Users\HP\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
app = Flask(__name__,template_folder='C:/Users/HP/3DEmotion/templates',static_folder='C:/Users/HP/3DEmotion/static/')
run_with_ngrok(app)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1
app.secret_key = 'sameer'
#!rm -rf 'C:/Users/HP/3DEmotion/static/output/'
@app.route('/')
def main():
    return render_template('index.html')

@app.route('/predict',  methods=['POST', 'GET'])
def uploadFile():
    if request.method == 'POST':
        if 'uploaded-file' not in request.files:
            return render_template('index.html')
        uploaded_img = request.files['uploaded-file']
        if uploaded_img.filename == '':
            return render_template('index.html')
        uploaded_img.save('C:/Users/HP/3DEmotion/static/file.jpg')
        img1 = cv2.imread('C:/Users/HP/3DEmotion/static/file.jpg')
        gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        cascade = cv2.CascadeClassifier('C:/Users/HP/3DEmotion/haarcascade_frontalface_default.xml')
        faces = cascade.detectMultiScale(gray, 1.1, 3)
        for x,y,w,h in faces:
            cv2.rectangle(img1, (x,y), (x+w, y+h), (0,255,0), 2)
            cropped = img1[y:y+h, x:x+w]
        cv2.imwrite('C:/Users/HP/3DEmotion/static/after.jpg', img1)
        try:
            cv2.imwrite('C:/Users/HP/3Demotion/static/cropped.jpg', cropped)
        except:
            pass
        try:
            image = cv2.imread('C:/Users/HP/3DEmotion/static/cropped.jpg', 0)
        except:
            image = cv2.imread('C:/Users/HP/3DEmotion/static/file.jpg', 0)
        image = tf.keras.utils.load_img('C:/Users/HP/3DEmotion/static/cropped.jpg',target_size = (48,48),color_mode = "grayscale")
        image = np.array(image)
        image = image/255.0
        image = np.reshape(image, (1,48,48,1))
        model = model_from_json(open("C:/Users/HP/3DEmotion/emotion_model1.json", "r").read())
        model.load_weights('C:/Users/HP/3DEmotion/model.h5')
        prediction = model.predict(image)
        label_dict = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}
        prediction = list(prediction[0])
        img_index = prediction.index(max(prediction))
        final_prediction=label_dict[img_index]
        imagePath = 'C:/Users/HP/3DEmotion/static/cropped.jpg'
        
        
        optimizer.run(imagePath)
        scene=a3d.Scene.from_file("C:/Users/HP/NextFace/static/output/mesh0.obj")
        scene.save("C:/Users/HP/NextFace/static/output/result.glTF")
        return render_template('predict.html', data=final_prediction)


@app.route('/contact')
def main2():
    return render_template('contact.html')

@app.route('/about')
def main3():
    return render_template('about.html')

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://063d-202-65-154-98.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Mar/2023 17:33:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /index1.css HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /static/index1.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /js/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /js/popper.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /js/jquery.validate.min.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /js/main.js HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:33:46] "GET /cdn-cgi/zaraz/s.js?z=JTdCJTIyZXhlY3V0ZWQlMjIlM0ElNUIlNUQlMkMlMjJ0JTIyJTNBJTIyQ29udGFjdCUyMEZvcm0lMjAwNCUyMiUyQyUyMnglMjIlM0EwLjg1MzI3NDQ3NjEwNzkzNjMlMkMlMjJ3JTIyJTNBMTI4MCUyQyUyMmglMjIlM0E3MjAlMkMlMjJqJTIyJTNBNjA5JTJDJTIyZSUyMiUzQTEyODAlMkMlMjJsJTIyJTNBJTIyaHR0cHMlM0ElMkYlMkZwcmV2aWV3LmNvbG9ybGliLmNvbSUyRnRoZW1lJTJGYm9vdHN0cmFwJTJGY29udGFjdC1mb3JtLTA0JTJGJTIyJTJDJTI

1/1 [==============================] - 0s 106ms/step
Loading Basel Face Model 2017 from C:/Users/HP/NextFace/baselMorphableModel//morphableModel-2017.pickle...
loading mesh normals...
loading uv parametrization...
loading landmarks association file...
creating sampler...
loading image from path:  C:/Users/HP/3DEmotion/static/cropped.jpg
detecting landmarks using: fan


init camera pose...
1/3 => Optimizing head pose and expressions using landmarks...
100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:59<00:00, 33.73it/s]
2/3 => Optimizing shape, statistical albedos, expression, head pose and scene light...
100%|████████████████████████████████████████████████████████████████████████████████| 401/401 [05:13<00:00,  1.28it/s]
3/3 => finetuning albedos, shape, expression, head pose and scene light...
100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [01:52<00:00,  1.11s/it]
took 8.10 minutes to optimize
saving to: ' C:/Users/HP/NextFace/static/output// '. hold on... 
127.0.0.1 - - [25/Mar/2023 17:44:06] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2023 17:44:06] "GET /index1.css HTTP/1.1" 404 -
127.0.0.1 - - [25/Mar/2023 17:44:06] "GET /static/index1.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Mar/2023 17:44:06] "GET /static/js/aframe.min.js HTTP/1.1" 404 -
12

In [4]:
!pip install torchvision